In [1]:
import pptx
import pandas as pd
import numpy as np
import glob
from pptx.enum.shapes import MSO_SHAPE_TYPE
from pptx.util import Inches
import copy
from pptx.shapes.autoshape import Shape

from pptx.shapes.graphfrm import GraphicFrame

from pptx.dml.color import RGBColor
from pptx.enum.dml import MSO_THEME_COLOR

from pptx.util import Inches,Pt
from pptx.enum.text import PP_ALIGN
from pptx.enum.dml import MSO_LINE
from pptx.enum.text import MSO_ANCHOR
from pptx.enum.dml import MSO_COLOR_TYPE, MSO_FILL
import re
import os

In [2]:
_EMUS_PER_INCH = 914400
_EMUS_PER_CENTIPOINT = 127
_EMUS_PER_CM = 360000
_EMUS_PER_MM = 36000
_EMUS_PER_PT = 12700

In [3]:
ppt_output = pptx.Presentation()

In [4]:
wbr_template = pptx.Presentation('Scenario2_template_new.pptx')

In [5]:
ppt_output.slide_width = wbr_template.slide_width
ppt_output.slide_height = wbr_template.slide_height

In [6]:
heading_path = 'C:\\Users\\vidhi.mittal\\Documents\\Automation\\MBR_3\\OPS_MBR_Github\\ops_mbr\\RX\\automation\\heading_Images\\'
image_replace_path = "C:\\Users\\vidhi.mittal\\Documents\\Automation\\MBR_3\\OPS_MBR_Github\\ops_mbr\\RX\\automation\\Scenario_2\\images\\"

In [7]:
""" create base template file """
writer = pd.ExcelWriter('excel.xlsx', engine='xlsxwriter')

for slide in wbr_template.slides:
    slide_no = (wbr_template.slides.index(slide))+1
    sheetname = "slide_{:02d}".format(slide_no)
    tdf = pd.DataFrame()
    
    for shape in slide.shapes:
        odf = pd.DataFrame([['','','','','','']],\
                           columns=['slide_no','shape_name','shape_type','shape_id','template_value','replace_value'])
        #odf = pd.DataFrame([])
        odf['slide_no'] = sheetname
        odf['shape_name'] = shape.name
        odf['shape_id'] = shape.shape_id
        
        if shape.shape_type == MSO_SHAPE_TYPE.PICTURE:
            odf['shape_type'] ='image'
            try:
                odf['template_value'] = copy.deepcopy(shape).element.xpath("//p:nvPicPr//p:cNvPr/@descr")[0] 
            except:
                odf['template_value'] = shape.name
        if shape.shape_type == MSO_SHAPE_TYPE.TEXT_BOX:
            odf['shape_type'] = 'text box'
            odf['template_value'] = shape.text
        
        tdf = tdf.append(odf)
    
    tdf.to_excel(writer,sheet_name=sheetname,index=False)

writer.save()

In [8]:
schemecolormap = {'accent1': MSO_THEME_COLOR.ACCENT_1,\
                 'accent2': MSO_THEME_COLOR.ACCENT_2,\
                  'accent3': MSO_THEME_COLOR.ACCENT_3,\
                  'accent4': MSO_THEME_COLOR.ACCENT_4,\
                  'accent5': MSO_THEME_COLOR.ACCENT_5,\
                  'accent6': MSO_THEME_COLOR.ACCENT_6,\
                  'bg1': MSO_THEME_COLOR.BACKGROUND_1,\
                  'bg2': MSO_THEME_COLOR.BACKGROUND_2,\
                  'dk1': MSO_THEME_COLOR.DARK_1,\
                  'dk2': MSO_THEME_COLOR.DARK_2,\
                  'lt1': MSO_THEME_COLOR.LIGHT_1,\
                  'lt2': MSO_THEME_COLOR.LIGHT_2,\
                  'tx1': MSO_THEME_COLOR.TEXT_1,\
                  'tx2': MSO_THEME_COLOR.TEXT_2
                 
                 }

In [9]:
paragraph_align_map = {
    "ctr": PP_ALIGN.CENTER,\
    "r": PP_ALIGN.RIGHT,
    "l": PP_ALIGN.LEFT,
    
}

In [10]:
box_border_dashmap = {
    
    "solid": MSO_LINE.SOLID,\
    "dot": MSO_LINE.ROUND_DOT,\
    "dash": MSO_LINE.DASH,\
    "lgDash": MSO_LINE.LONG_DASH,\
    "dashDot": MSO_LINE.DASH_DOT,\
    "lgDashDot": MSO_LINE.LONG_DASH_DOT,\
    "lgDashDotDot": MSO_LINE.DASH_DOT_DOT,\
    "sysDash": MSO_LINE.DASH,\
    "sysDot": MSO_LINE.SQUARE_DOT,\
    "sysDashDot": MSO_LINE.DASH_DOT,\
    "sysDashDotDot": MSO_LINE.DASH_DOT_DOT,\
    
    
}

In [11]:
def get_rgb_color(color_string):

    rcol = int(color_string[0:2],16)
    bcol = int(color_string[2:4],16)
    gcol = int(color_string[4:6],16)
    
    return RGBColor(rcol,bcol,gcol)

In [12]:
def get_textbox_properties(sph):
    #print(sph.element.xml)
    fillcolor_schemecolor = (sph.element.xpath("//p:spPr/a:solidFill//a:schemeClr/@val"))
    fillcolor_rbgcolor = (sph.element.xpath("//p:spPr/a:solidFill//a:srgbClr/@val"))
    fontsz = (sph.element.xpath("//a:rPr/@sz"))
    fontface = (sph.element.xpath("//a:rPr//@typeface"))
    fillbrightness = sph.element.xpath("//p:spPr/a:solidFill//a:schemeClr//a:lumOff//@val")
    filldarkness = sph.element.xpath("//p:spPr/a:solidFill//a:schemeClr//a:lumMod//@val")
    
    font_isbold = (sph.element.xpath("//a:rPr/@b"))
    font_isitalic = (sph.element.xpath("//a:rPr/@i"))
    font_isunderline = (sph.element.xpath("//a:rPr/@u"))
    font_iscentered = (sph.element.xpath("//a:rPr/a:cs"))
    font_alignment = (sph.element.xpath("//p:txBody//a:p//a:pPr/@algn"))
    
    box_border_schemecolor = (sph.element.xpath("//p:spPr//a:ln//a:solidFill//a:schemeClr/@val"))
    box_border_rbgcolor = (sph.element.xpath("//p:spPr//a:ln//a:solidFill//a:srgbClr/@val"))
    box_border_width = (sph.element.xpath("//p:spPr//a:ln/@w"))
    box_border_brightness = (sph.element.xpath("//p:spPr//a:ln//a:solidFill//a:schemeClr//a:lumOff/@val"))
    box_border_darkness = (sph.element.xpath("//p:spPr//a:ln//a:solidFill//a:schemeClr//a:lumMod/@val"))
    box_border_isdash = (sph.element.xpath("//p:spPr//a:ln//a:prstDash/@val"))
    
    
    if (len(fillbrightness)==0)&(len(filldarkness)!=0):
        fillbrightness = [100000-int(filldarkness[0])]
    
    if (len(box_border_brightness)==0)&(len(box_border_darkness)!=0):
        box_border_brightness = [100000-int(box_border_darkness[0])]
       
    if len(font_iscentered)!=0:
        palign = paragraph_align_map.get("ctr")
    else:
        try:
            palign = paragraph_align_map.get(font_alignment[0])
        except:
            palign = paragraph_align_map.get('r')
    
    #print(sph.fill.fore_color.type==MSO_COLOR_TYPE.SCHEME)
#     print('$$$$$$$$$$$$$$$$$$$$$$$$',fillcolor_schemecolor,fillcolor_rbgcolor,fillbrightness,font_alignment,box_border_schemecolor,box_border_rbgcolor,box_border_brightness)
    outdict = {
        
        #"fillcolor_schemecolor": schemecolormap.get(fillcolor_schemecolor[0]) if len(fillcolor_schemecolor)!=0 else None,\
        "fillcolor_schemecolor": (sph.fill.fore_color.theme_color) if len(fillcolor_schemecolor)!=0 else None,\
        "fillcolor_rbgcolor": get_rgb_color(fillcolor_rbgcolor[0]) if len(fillcolor_rbgcolor)!=0 else None,\
        "fontsz": (int(fontsz[0])/100) if len(fontsz)!=0 else None,\
        "fontface": fontface[0] if len(fontface)!=0 else None,\
        "fillbrightness": (int(fillbrightness[0])/100000) if len(fillbrightness)!=0 else None,\
        
        "font_isbold": False if(len(font_isbold)==0 or font_isbold[0]!='1') else True,\
        "font_isitalic": False if (len(font_isitalic)==0 or font_isitalic[0]!='1')!=0 else True,\
        "font_isunderline": False if (len(font_isunderline)==0 or font_isunderline[0]!='1')!=0 else True,\
        
        "font_alignment": palign,\
        
        "box_border_schemecolor": schemecolormap.get(box_border_schemecolor[0]) if len(box_border_schemecolor)!=0 else None,\
        "box_border_rbgcolor": get_rgb_color(box_border_rbgcolor[0]) if len(box_border_rbgcolor)!=0 else None,\
        "box_border_brightness": (int(box_border_brightness[0])/100000) if len(box_border_brightness)!=0 else None,\
        "box_border_width" : (int(box_border_width[0])/_EMUS_PER_PT) if len(box_border_width)!=0 else None,\
        "box_border_isdash":box_border_dashmap.get(box_border_isdash[0]) if len(box_border_isdash)!=0 else None
        
        
    }
    
    return outdict
    

In [13]:
def set_textbox_properties(tb,prop_dict):
    print(prop_dict)
    tf = tb.text_frame
    p = tf.paragraphs[0]
    run =p.add_run()
    #run.text = shape.text
    if prop_dict["fillcolor_schemecolor"]:
        tb.fill.solid()
        print(prop_dict["fillcolor_schemecolor"])
        tb.fill.fore_color.theme_color = prop_dict["fillcolor_schemecolor"]
    
    if prop_dict["fillcolor_rbgcolor"]:
        tb.fill.solid()
        tb.fill.fore_color.rgb = prop_dict["fillcolor_rbgcolor"]
        
    if prop_dict["fillbrightness"]:
        tb.fill.fore_color.brightness = prop_dict["fillbrightness"]
        #pass
    
    if prop_dict["fontsz"]:
        run.font.size=Pt(prop_dict["fontsz"])
        
    if prop_dict["fontface"]:
        run.font.name =prop_dict["fontface"]
    
    
    run.font.bold = prop_dict["font_isbold"]
    run.font.italic = prop_dict["font_isitalic"]
    run.font.underline = prop_dict["font_isunderline"]
    
    p.alignment = prop_dict["font_alignment"]
    
    
    if prop_dict["box_border_schemecolor"]:
        tb.line.color.theme_color = prop_dict["box_border_schemecolor"]
        
    if prop_dict["box_border_rbgcolor"]:
        tb.line.color.rgb = prop_dict["box_border_rbgcolor"]
    
    if prop_dict["box_border_brightness"]:
        #pass
        tb.line.color.brightness = prop_dict["box_border_brightness"]
    
    if prop_dict["box_border_width"]:
        tb.line.width = Pt(prop_dict["box_border_width"])
        
    if prop_dict["box_border_isdash"]:
        tb.line.dash_style = prop_dict["box_border_isdash"]
    
    
        
    return run

In [14]:
def set_text_holders(stext,text_replace_dict):
    otext = str(stext)
    sgr = (re.findall(r'{{(.*?)}}',otext))
       
    for sl in sgr:
        print(sl)
        print(str(text_replace_dict.get(sl)))
        otext = (otext).replace(sl,str(text_replace_dict.get(sl)))\
            .replace("{","").replace("}","")
    
    return otext   

In [15]:
total_revenue = 7.17
rx_count = 234
active_provider = 100
patient_served = 678
new_provider = 897
rev_per_rx = 23

In [16]:
def get_variable_names(stext):
    otext = str(stext)
    sgr = (re.findall(r'{{(.*?)}}',otext))
    return sgr

In [17]:
def create_image_dict(df):
    xdf = df[['shape_name','replace_value']]
    return xdf.set_index('shape_name').to_dict()["replace_value"]

In [18]:
def create_variable_dict(df):
    xdf = df[['template_value']]
    
    hlist = []
    for i,r in xdf.iterrows():
        
        xlist = get_variable_names(r['template_value'])
        hlist = hlist + xlist
    
    vdict = {}
    for x in np.unique(np.array(hlist)):
        vdict[x] = eval(x)
    return vdict
    

In [19]:
variable_template_df = pd.ExcelFile('final_excel.xlsx')

In [20]:
for slide in wbr_template.slides:
    slide_no = (wbr_template.slides.index(slide))+1
    sheetname = "slide_{:02d}".format(slide_no)
    tslide = slide
    #print(tslide)
    oslide = ppt_output.slides.add_slide(ppt_output.slide_layouts[6])
    vtdf = variable_template_df.parse(sheetname)
    image_replace_dict = create_image_dict(vtdf[vtdf['shape_type']=='image'])
    text_replace_dict = create_variable_dict(vtdf[vtdf['shape_type']=='text box'])
    print(image_replace_dict)
    for shape in tslide.shapes:
        #print(shape._element.xml)
        #print(shape.name)
        sleft = Inches(shape.left/_EMUS_PER_INCH)
        swidth = Inches(shape.width/_EMUS_PER_INCH)
        sheight = Inches(shape.height/_EMUS_PER_INCH)
        stop = Inches(shape.top/_EMUS_PER_INCH)

        if shape.shape_type == MSO_SHAPE_TYPE.PICTURE:
            img_rep = image_replace_dict.get(shape.name)
            try:
                img_path = image_replace_path+img_rep
                head = heading_path+img_rep
                if os.path.exists(img_path):
                    oslide.shapes.add_picture(img_path,left=sleft,top=stop,width=swidth,height=sheight)
                elif os.path.exists(head):
                    oslide.shapes.add_picture(head,left=sleft,top=stop,width=swidth,height=sheight)
            except:
                pass


        if shape.shape_type == MSO_SHAPE_TYPE.TEXT_BOX:
            #print(shape.shape_id)

            cshape = copy.deepcopy(shape)
            property_dict = (get_textbox_properties(cshape))

            tb = oslide.shapes.add_textbox(left=sleft,top=stop,width=swidth,height=sheight)
            
            run = set_textbox_properties(tb,property_dict)

            run.text = (set_text_holders(shape.text,text_replace_dict))
            print(run.text)

    ppt_output.save("C:\\Users\\vidhi.mittal\\Documents\\Automation\\MBR_3\\OPS_MBR_Github\\ops_mbr\\RX\\merging\\All_ppts\\F_Scenerio_2_ppt.pptx")
       

{'Picture 2': 'heading_median_scen-2.png'}
{'fillcolor_schemecolor': None, 'fillcolor_rbgcolor': None, 'fontsz': None, 'fontface': None, 'fillbrightness': None, 'font_isbold': False, 'font_isitalic': False, 'font_isunderline': False, 'font_alignment': 3, 'box_border_schemecolor': None, 'box_border_rbgcolor': None, 'box_border_brightness': None, 'box_border_width': None, 'box_border_isdash': None}

{'Picture 3': 'split_scenario_2_df2_overall.png', 'Picture 4': 'split_scenario_2_df1_overall.png'}
{'fillcolor_schemecolor': None, 'fillcolor_rbgcolor': None, 'fontsz': None, 'fontface': None, 'fillbrightness': None, 'font_isbold': False, 'font_isitalic': False, 'font_isunderline': False, 'font_alignment': 3, 'box_border_schemecolor': None, 'box_border_rbgcolor': None, 'box_border_brightness': None, 'box_border_width': None, 'box_border_isdash': None}

{'fillcolor_schemecolor': None, 'fillcolor_rbgcolor': None, 'fontsz': 9.0, 'fontface': 'Nunito', 'fillbrightness': None, 'font_isbold': False,

In [21]:
oslide.shapes[0].element.xml

'<p:pic xmlns:p="http://schemas.openxmlformats.org/presentationml/2006/main" xmlns:a="http://schemas.openxmlformats.org/drawingml/2006/main" xmlns:r="http://schemas.openxmlformats.org/officeDocument/2006/relationships">\n  <p:nvPicPr>\n    <p:cNvPr id="2" name="Picture 1" descr="MD-BA-rph_verified_to_delivered_refill.png"/>\n    <p:cNvPicPr>\n      <a:picLocks noChangeAspect="1"/>\n    </p:cNvPicPr>\n    <p:nvPr/>\n  </p:nvPicPr>\n  <p:blipFill>\n    <a:blip r:embed="rId2"/>\n    <a:stretch>\n      <a:fillRect/>\n    </a:stretch>\n  </p:blipFill>\n  <p:spPr>\n    <a:xfrm>\n      <a:off x="343456" y="2931543"/>\n      <a:ext cx="3424196" cy="1835720"/>\n    </a:xfrm>\n    <a:prstGeom prst="rect">\n      <a:avLst/>\n    </a:prstGeom>\n  </p:spPr>\n</p:pic>\n'